In [1]:
! pip install --quiet transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.1 MB/s eta 0:00:00


In [2]:
import torch
from PIL import Image
from datasets import load_dataset
from transformers import CLIPProcessor, CLIPModel
from tqdm.auto import tqdm
import numpy as np

In [3]:
device = torch.device("cuda:0")

In [4]:
# load dataset
dataset = load_dataset("jxie/flickr8k")

# load model
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

model = model.to(device)

Generating train split:   0%|          | 0/6000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [5]:
img_embeds = []
for batch in tqdm(dataset['train'], total=len(dataset['train'])):
    with torch.no_grad():
        inputs = processor(images=batch['image'], return_tensors="pt")
        inputs = inputs.to(device)
        image_features = model.get_image_features(**inputs) # 1 x 512
        img_embeds.append(image_features[0].cpu().numpy())

  0%|          | 0/6000 [00:00<?, ?it/s]

In [6]:
np.stack(img_embeds).shape

(6000, 512)

In [7]:
np.save("image_embeds.npy", img_embeds)